# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [25]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv



#### Creating list of filepaths to process original event csv data files

In [26]:
# checking current working directory
print(os.getcwd())

# Get  current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [27]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [28]:
# checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion. 

##  ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId



##  code in the cells below

#### Creating a Cluster

In [29]:
#  making a connection to a Cassandra instance on local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)


#establishing a connection to begin executing queries, making a session session


#### Creating a Keyspace

In [30]:
#Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [31]:
#Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Solution Query 1 

In [32]:
# making a Table that will Query 
# the artist, song title and song's length in the music app history
# that was heard during sessionId = 338, and itemInSession = 4


#====================================================
# Creating table song_length
# the userId and sessionId was used as a composite partition key 
# because the queries will filter by these columns. 
# The itemInSession were used as clustering column 
# to help make up a unique key.
#====================================================


#========
# creating the table 
#========

session.execute( """CREATE TABLE IF NOT EXISTS song_length 
(sessionid int,iteminsession int ,artist text , song text, length double
, PRIMARY KEY ( sessionid, iteminsession ))""") 


#==========
# inserting tha data 
#===========
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:

        query = "INSERT INTO song_length  (artist, song , length, sessionid , iteminsession)"
        query = query + "VALUES (%s, %s, %s, %s ,%s)"
        
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#==========
# checking the data 
#==========
        
query = "select * from song_length  WHERE sessionId = 338 and itemInSession = 4"

## geting the artist, song title and song's length in the music app history
## that was heard during sessionId = 338, and itemInSession = 4

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (f' artist: {row.artist}, song: {row.song}, lenght: {row.length}')        

 artist: Faithless, song: Music Matters (Mark Knight Dub), lenght: 495.3073


# Solution Query 2

In [18]:
# making a Table that will Query 
# name of artist, song (sorted by itemInSession) and user (first and last name) 
# for userid = 10, sessionid = 182


#====================================================
# Creating table artist_song
# the userId and sessionId was used as a composite partition key 
# because the queries will filter by these columns. 
# The itemInSession were used as clustering column 
# to help make up a unique key.
#====================================================




#========
# creating the table 
#========

session.execute("""CREATE TABLE IF NOT EXISTS artist_song 
(userId int ,sessionid int, iteminsession int ,artist text, firstname text  , song text, lastname text,  
PRIMARY KEY ((userId, sessionId), itemInSession)) """)

#==========
# inserting tha data 
#===========
file = 'event_datafile_new.csv'    

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        
        query = "INSERT INTO artist_song  (artist, song, firstname, lastname , userId, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s ,%s ,%s ,%s)"
        
    
        session.execute(query , (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])) )
          
#==========
# checking the data 
#==========


query = "select artist, song, firstname, lastname from artist_song WHERE userId= 10 and sessionId = 182"

## getting name of artist, 
## song (sorted by itemInSession) and user (first and last name) 
## for userid = 10, sessionid = 182

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
       print(f'artist: {row.artist}, song: {row.song}, user first name: {row.firstname},user last name: {row.lastname}')

artist: Down To The Bone, song: Keep On Keepin' On, user first name: Sylvie,user last name: Cruz
artist: Three Drives, song: Greece 2000, user first name: Sylvie,user last name: Cruz
artist: Sebastien Tellier, song: Kilometer, user first name: Sylvie,user last name: Cruz
artist: Lonnie Gordon, song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), user first name: Sylvie,user last name: Cruz


# Solution Query 3

In [37]:
# making a Table that wil Query 
# every user name (first and last) in my music app history who
#listened to the song 'All Hands Against His Own' 

#===============================
# creating song_listeners 
# where song table is the primary key 
# because we will be filtering by this colums 
# and userid as partitionkey 
# to make a unique Key
#================================


#=========
# creating Table 
#===========

session.execute("""CREATE TABLE IF NOT EXISTS song_listeners 
(song text, userid  int ,first_name text, last_name text, PRIMARY KEY(song, userid));""")

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_listeners (song, userid , first_name, last_name)"  
        query = query + "VALUES (%s, %s, %s, %s) IF NOT EXISTS "
        session.execute(query, (line[9], int(line[10]), line[1], line[4]) )
        
    
query =  "SELECT first_name, last_name FROM song_listeners WHERE song = 'All Hands Against His Own'"

## getting every user name (first and last) in my music app history who
## listened to the song 'All Hands Against His Own' 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
for row in rows :
    print (f'User Name: {row.first_name} {row.last_name}')

User Name: Jacqueline Lynch
User Name: Tegan Levine
User Name: Sara Johnson


### Drop the tables before closing out the sessions

In [47]:
session.execute(" DROP TABLE   song_length " )
session.execute(" DROP TABLE   artist_song" )
session.execute(" DROP TABLE   song_listeners" )


### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()